In [4]:
import string
import statistics
import nltk
import spacy
import matplotlib
import os
import string
import warnings
import numpy as np
import pandas as pd
from scipy import stats
from joblib import parallel_backend
from sklearn import linear_model, decomposition
from collections import OrderedDict
from operator import itemgetter
from sklearn import metrics
from joblib import Parallel, delayed
from sklearn.model_selection import GridSearchCV, cross_validate, cross_val_score, StratifiedKFold, RepeatedStratifiedKFold, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler, FunctionTransformer, OneHotEncoder, LabelEncoder
from sklearn.feature_selection import f_classif, VarianceThreshold, SelectFwe
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin, clone, RegressorMixin
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.naive_bayes import BernoulliNB
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score

try:
    from sklearn.utils._testing import ignore_warnings
except:
    from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

In [9]:
def slice_by_corr(X, r_min=0.25):
    # Create correlation matrix
    corr_matrix = X.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # Find features with correlation greater than r_min
    return X[[column for column in upper.columns if any(upper[column] > r_min)]]

def variance_inflation_factor(X, exog_idx):
    clf = LinearRegression(fit_intercept=True)
    sub_X = np.delete(np.nan_to_num(X), exog_idx, axis=1)
    sub_y = X[:, exog_idx][np.newaxis].T
    sub_clf = clf.fit(sub_X, sub_y)
    return 1 / (1 - r2_score(sub_y, sub_clf.predict(sub_X)))

class ReduceVIF(BaseEstimator, TransformerMixin):

    def __init__(self, thresh=10.0, nthreads=4, r_min=0.25, obs=250):
        self.thresh = thresh
        self.nthreads = nthreads
        self.r_min = r_min
        self.obs = obs
        
    def fit(self, X):
        self.X = X
        return self

    def transform(self, X):
        return ReduceVIF.calculate_vif(X, self.thresh, 
                                       self.nthreads, 
                                       self.r_min, 
                                       self.obs)

    @staticmethod
    def calculate_vif(X, thresh=10.0, nthreads=16, r_min=0.25, obs=250):        
        dropped = True
        vif_cols = []
        X_vif_candidates = slice_by_corr(X, r_min)
        X_vif_candidates = X_vif_candidates.sample(n=obs)
        while dropped:
            variables = X_vif_candidates.columns
            dropped = False
            with Parallel(n_jobs=nthreads, backend='threading') as parallel:
                vif = parallel(
                    delayed(variance_inflation_factor)(
                        np.asarray(X_vif_candidates[variables].values),
                        X_vif_candidates.columns.get_loc(var)) for var in 
                    X_vif_candidates.columns)
            max_vif = max(vif)
            if max_vif > thresh:
                maxloc = vif.index(max_vif)
                print(f'Dropping {X_vif_candidates.columns[maxloc]} with vif={max_vif}')
                vif_cols.append(X_vif_candidates.columns.tolist()[maxloc])
                X_vif_candidates = X_vif_candidates.drop(
                    [X_vif_candidates.columns.tolist()[maxloc]], axis=1)
                dropped = True
        
        return X[[~vif_cols]], vif_cols

    
def preprocess_x_y(X, nodrop_columns=[],
                   var_thr=0.80, remove_multi=True,
                   standardize=True,
                   std_dev=3, vif_thr=5, missingness_thr=0.50,
                   zero_thr=0.50, nthreads=16):
    from colorama import Fore, Style

    # Replace all near-zero with zeros
    # Drop excessively sparse columns with >zero_thr zeros
    if zero_thr > 0:
        X = X.apply(lambda x: np.where(np.abs(x) < 0.000001, 0, x))
        X_tmp = X.T.loc[(X == 0).sum() < (float(zero_thr)) * X.shape[0]].T

        if len(nodrop_columns) > 0:
            X = pd.concat([X_tmp, X[[i for i in X.columns if i in
                                     nodrop_columns and i not in
                                     X_tmp.columns]]], axis=1)
        else:
            X = X_tmp
        del X_tmp

        if X.empty or len(X.columns) < 5:
            print(f"\n\n{Fore.RED}Empty feature-space (Zero Columns): "
                  f"{X}{Style.RESET_ALL}\n\n")
            return X

    # Remove columns with excessive missing values
    X = X.dropna(thresh=len(X) * (1 - missingness_thr), axis=1)
    if X.empty:
        print(f"\n\n{Fore.RED}Empty feature-space (missingness): "
              f"{X}{Style.RESET_ALL}\n\n")
        return X

    # Apply a simple imputer (note that this assumes extreme cases of
    # missingness have already been addressed). The SimpleImputer is better
    # for smaller datasets, whereas the IterativeImputer performs best on
    # larger sets.

    # from sklearn.experimental import enable_iterative_imputer
    # from sklearn.impute import IterativeImputer
    # imp = IterativeImputer(random_state=0, sample_posterior=True)
    # X = pd.DataFrame(imp.fit_transform(X, y), columns=X.columns)
    imp1 = SimpleImputer()
    X = pd.DataFrame(imp1.fit_transform(X.astype('float32')),
                     columns=X.columns)

    # Standardize X
    if standardize is True:
        scaler = StandardScaler()
        X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

    # Remove low-variance columns
    sel = VarianceThreshold(threshold=(var_thr * (1 - var_thr)))
    sel.fit(X)
    if len(nodrop_columns) > 0:
        good_var_cols = X.columns[np.concatenate(
            [sel.get_support(indices=True), np.array([X.columns.get_loc(c)
                                                      for c in
                                                      nodrop_columns if
                                                      c in X])])]
    else:
        good_var_cols = X.columns[sel.get_support(indices=True)]

    low_var_cols = [i for i in X.columns if i not in list(good_var_cols)]
    if len(low_var_cols) > 0:
        print(f"Dropping {low_var_cols} for low variance...")
    X = X[good_var_cols]

    if X.empty:
        print(f"\n\n{Fore.RED}Empty feature-space (low-variance): "
              f"{X}{Style.RESET_ALL}\n\n")
        return X

    # Remove multicollinear columns
    if remove_multi is True:
        try:
            rvif = ReduceVIF(thresh=vif_thr, nthreads=nthreads)
            X = rvif.fit_transform(X)[0]
            if X.empty or len(X.columns) < 5:
                print(f"\n\n{Fore.RED}Empty feature-space "
                      f"(multicollinearity): "
                      f"{X}{Style.RESET_ALL}\n\n")
                return X
        except:
            print(f"\n\n{Fore.RED}Empty feature-space (multicollinearity): "
                  f"{X}{Style.RESET_ALL}\n\n")
            return X

    print(f"\nX: {X}\n")
    print(f"Features: {list(X.columns)}\n")
    return X


class Razors(object):
    """
    Razors is a callable refit option for `GridSearchCV` whose aim is to
    balance model complexity and cross-validated score in the spirit of the
    "one standard error" rule of Breiman et al. (1984), which showed that
    the tuning hyperparameter associated with the best performing model may be
    prone to overfit. To help mitigate this risk, we can instead instruct
    gridsearch to refit the highest performing 'parsimonious' model, as defined
    using simple statistical rules (e.g. standard error (`sigma`),
    percentile (`eta`), or significance level (`alpha`)) to compare
    distributions of model performance across folds. Importantly, this
    strategy assumes that the grid of multiple cross-validated models
    can be principly ordered from simplest to most complex with respect to some
    target hyperparameter of interest. To use the razors suite, supply
    the `simplify` function partial of the `Razors` class as a callable
    directly to the `refit` argument of `GridSearchCV`.

    Parameters
    ----------
    cv_results : dict of numpy(masked) ndarrays
        See attribute cv_results_ of `GridSearchCV`.
    scoring : str
        Refit scoring metric.
    param : str
        Parameter whose complexity will be optimized.
    rule : str
        Rule for balancing model complexity with performance.
        Options are 'se', 'percentile', and 'ranksum'. Default is 'se'.
    sigma : int
        Number of standard errors tolerance in the case that a standard error
        threshold is used to filter outlying scores across folds. Required if
        `rule`=='se'. Default is 1.
    eta : float
        Percentile tolerance in the case that a percentile threshold
        is used to filter outlier scores across folds. Required if
        `rule`=='percentile'. Default is 0.68.
    alpha : float
        An alpha significance level in the case that wilcoxon rank sum
        hypothesis testing is used to filter outlying scores across folds.
        Required if `rule`=='ranksum'. Default is 0.05.

    References
    ----------
    Breiman, Friedman, Olshen, and Stone. (1984) Classification and Regression
    Trees. Wadsworth.

    Notes
    -----
    Here, 'simplest' is defined by the complexity of the model as influenced by
    some user-defined target parameter (e.g. number of components, number of
    estimators, polynomial degree, cost, scale, number hidden units, weight
    decay, number of nearest neighbors, L1/L2 penalty, etc.).

    The callable API accordingly assumes that the `params` attribute of
    `cv_results_` 1) contains the indicated hyperparameter (`param`) of
    interest, and 2) contains a sequence of values (numeric, boolean, or
    categorical) that are ordered from least to most complex.
    """
    __slots__ = ('cv_results', 'param', 'param_complexity', 'scoring',
                 'rule', 'greater_is_better',
                 '_scoring_funcs', '_scoring_dict',
                 '_n_folds', '_splits', '_score_grid',
                 '_cv_means', '_sigma', '_eta', '_alpha')

    def __init__(
            self,
            cv_results_,
            param,
            scoring,
            rule,
            sigma=1,
            eta=0.95,
            alpha=0.01,
    ):
        import sklearn.metrics

        self.cv_results = cv_results_
        self.param = param
        self.scoring = scoring
        self.rule = rule
        self._scoring_funcs = [
            met
            for met in sklearn.metrics.__all__
            if (met.endswith("_score")) or (met.endswith("_error"))
        ]
        # Set _score metrics to True and _error metrics to False
        self._scoring_dict = dict(
            zip(
                self._scoring_funcs,
                [met.endswith("_score") for met in self._scoring_funcs],
            )
        )
        self.greater_is_better = self._check_scorer()
        self._n_folds = len(list(set([i.split('_')[0] for i in
                                     list(self.cv_results.keys()) if
                                     i.startswith('split')])))
        # Extract subgrid corresponding to the scoring metric of interest
        self._splits = [i for i in list(self.cv_results.keys()) if
                        i.endswith(f"test_{self.scoring}") and
                        i.startswith('split')]
        self._score_grid = np.vstack([self.cv_results[cv] for cv in
                                      self._splits]).T
        self._cv_means = np.array(np.nanmean(self._score_grid, axis=1))
        self._sigma = sigma
        self._eta = eta
        self._alpha = alpha

    def _check_scorer(self):
        """
        Check whether the target refit scorer is negated. If so, adjust
        greater_is_better accordingly.
        """

        if (
                self.scoring not in self._scoring_dict.keys()
                and f"{self.scoring}_score" not in self._scoring_dict.keys()
        ):
            if self.scoring.startswith("neg_"):
                self.greater_is_better = True
            else:
                raise NotImplementedError(f"Scoring metric {self.scoring} not "
                                          f"recognized.")
        else:
            self.greater_is_better = [
                value for key, value in self._scoring_dict.items() if
                self.scoring in key][0]
        return self.greater_is_better

    def _best_low_complexity(self):
        """
        Balance model complexity with cross-validated score.

        Return
        ------
        int
            Index of a model that has the lowest complexity but its test score
            is the highest on average across folds as compared to other models
            that are equally likely to occur.
        """

        # Check parameter(s) whose complexity we seek to restrict
        if not any(self.param in x for x in
                   self.cv_results["params"][0].keys()):
            raise KeyError(f"Parameter {self.param} not found in cv grid.")
        else:
            hyperparam = [
                i for i in self.cv_results["params"][0].keys() if
                i.endswith(self.param)][0]

        # Select low complexity threshold based on specified evaluation rule
        if self.rule == "se":
            if not self._sigma:
                raise ValueError(
                    "For `se` rule, the tolerance "
                    "(i.e. `_sigma`) parameter cannot be null."
                )
            l_cutoff, h_cutoff = self.call_standard_error()
        elif self.rule == "percentile":
            if not self._eta:
                raise ValueError(
                    "For `percentile` rule, the tolerance "
                    "(i.e. `_eta`) parameter cannot be null."
                )
            l_cutoff, h_cutoff = self.call_percentile()
        elif self.rule == "ranksum":
            if not self._alpha:
                raise ValueError(
                    "For `ranksum` rule, the alpha-level "
                    "(i.e. `_alpha`) parameter cannot be null."
                )
            l_cutoff, h_cutoff = self.call_rank_sum_test()
        else:
            raise NotImplementedError(f"{self.rule} is not a valid "
                                      f"rule of RazorCV.")

        self.cv_results[f"param_{hyperparam}"].mask = np.where(
            (self._cv_means >= float(l_cutoff)) &
            (self._cv_means <= float(h_cutoff)),
            True, False)

        if np.sum(self.cv_results[f"param_{hyperparam}"].mask) == 0:
            print(f"\nLow: {l_cutoff}")
            print(f"High: {h_cutoff}")
            print(f"{self._cv_means}")
            print(f"hyperparam: {hyperparam}\n")
            raise ValueError("No valid grid columns remain within the "
                             "boundaries of the specified razor")

        highest_surviving_rank = np.nanmin(
            self.cv_results[f"rank_test_{self.scoring}"][
                self.cv_results[f"param_{hyperparam}"].mask])

        # print(f"Highest surviving rank: {highest_surviving_rank}\n")

        return np.flatnonzero(np.isin(
            self.cv_results[f"rank_test_{self.scoring}"],
            highest_surviving_rank))[0]

    def call_standard_error(self):
        """
        Returns the simplest model whose performance is within `sigma`
        standard errors of the average highest performing model.
        """

        # Estimate the standard error across folds for each column of the grid
        cv_se = np.array(np.nanstd(self._score_grid, axis=1) /
                         np.sqrt(self._n_folds))

        # Determine confidence interval
        if self.greater_is_better:
            best_score_idx = np.nanargmax(self._cv_means)
            h_cutoff = self._cv_means[best_score_idx] + cv_se[best_score_idx]
            l_cutoff = self._cv_means[best_score_idx] - cv_se[best_score_idx]
        else:
            best_score_idx = np.nanargmin(self._cv_means)
            h_cutoff = self._cv_means[best_score_idx] - cv_se[best_score_idx]
            l_cutoff = self._cv_means[best_score_idx] + cv_se[best_score_idx]

        return l_cutoff, h_cutoff

    def call_rank_sum_test(self):
        """
        Returns the simplest model whose paired performance across folds is
        insignificantly different from the average highest performing,
        at a predefined `alpha` level of significance.
        """

        from scipy.stats import wilcoxon
        import itertools

        if self.greater_is_better:
            best_score_idx = np.nanargmax(self._cv_means)
        else:
            best_score_idx = np.nanargmin(self._cv_means)

        # Perform signed Wilcoxon rank sum test for each pair combination of
        # columns against the best average score column
        tests = [pair for pair in list(itertools.combinations(range(
            self._score_grid.shape[0]), 2)) if best_score_idx in pair]

        p_dict = {}
        for i, test in enumerate(tests):
            p_dict[i] = wilcoxon(self._score_grid[test[0], :],
                                 self._score_grid[test[1], :])[1]

        # Sort and prune away significant tests
        p_dict = {k: v for k, v in sorted(p_dict.items(),
                                          key=lambda item: item[1]) if
                  v > self._alpha}

        # Flatten list of tuples, remove best score index, and take the
        # lowest and highest remaining bounds
        tests = [j for j in list(set(list(sum([tests[i] for i in
                                               list(p_dict.keys())],
                                              ())))) if j != best_score_idx]
        if self.greater_is_better:
            h_cutoff = self._cv_means[
                np.nanargmin(self.cv_results[
                                 f"rank_test_{self.scoring}"][tests])]
            l_cutoff = self._cv_means[
                np.nanargmax(self.cv_results[
                                 f"rank_test_{self.scoring}"][tests])]
        else:
            h_cutoff = self._cv_means[
                np.nanargmax(self.cv_results[
                                 f"rank_test_{self.scoring}"][tests])]
            l_cutoff = self._cv_means[
                np.nanargmin(self.cv_results[
                                 f"rank_test_{self.scoring}"][tests])]

        return l_cutoff, h_cutoff


    def call_percentile(self):
        """
        Returns the simplest model whose performance is within the `eta`
        percentile of the average highest performing model.
        """

        # Estimate the indicated percentile, and its inverse, across folds for
        # each column of the grid
        perc_cutoff = np.nanpercentile(self._score_grid,
                                       [100 * self._eta,
                                        100 - 100 * self._eta], axis=1)

        # Determine bounds of the percentile interval
        if self.greater_is_better:
            best_score_idx = np.nanargmax(self._cv_means)
            h_cutoff = perc_cutoff[0, best_score_idx]
            l_cutoff = perc_cutoff[1, best_score_idx]
        else:
            best_score_idx = np.nanargmin(self._cv_means)
            h_cutoff = perc_cutoff[0, best_score_idx]
            l_cutoff = perc_cutoff[1, best_score_idx]

        return l_cutoff, h_cutoff

    @staticmethod
    def simplify(param, scoring, rule='se', sigma=1, eta=0.68, alpha=0.01):
        """
        Callable to be run as `refit` argument of `GridsearchCV`.

        Parameters
        ----------
        param : str
            Parameter with the largest influence on model complexity.
        scoring : str
            Refit scoring metric.
        sigma : int
            Number of standard errors tolerance in the case that a standard
            error threshold is used to filter outlying scores across folds.
            Only applicable if `rule`=='se'. Default is 1.
        eta : float
            Acceptable percent tolerance in the case that a percentile
            threshold is used. Only applicable if `rule`=='percentile'.
            Default is 0.68.
        alpha : float
            Alpha-level to use for signed wilcoxon rank sum testing.
            Only applicable if `rule`=='ranksum'. Default is 0.01.
        """
        from functools import partial

        def razor_pass(
                cv_results_, param, scoring, rule, sigma, alpha, eta
        ):
            rcv = Razors(cv_results_, param, scoring, rule=rule,
                         sigma=sigma, alpha=alpha, eta=eta)
            return rcv._best_low_complexity()

        return partial(
            razor_pass,
            param=param,
            scoring=scoring,
            rule=rule,
            sigma=sigma,
            alpha=alpha,
            eta=eta,
        )

def divide_df(df_all,train_len):
    return df_all.loc[:train_len-1], df_all.loc[train_len:].drop('target',axis=1)

# Load data

In [6]:
#working_dir = '/kaggle/input/nlp-getting-started/'
working_dir = '/home/dpys/Documents/Kaggle_Competitions/nlp-getting-started/'
df_train_all = pd.read_csv(f"{working_dir}train.csv")
df_test_all = pd.read_csv(f"{working_dir}test.csv")
print(f"Rows in train.csv = {len(df_train_all)}")
print(f"Rows in test.csv = {len(df_test_all)}")
pd.set_option('display.max_colwidth', None)
df_train_all.head()

Rows in train.csv = 7613
Rows in test.csv = 3263


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


# Exploratory Data Analysis

In [7]:
df_train_pos = df_train_all[df_train_all.target == 1]
df_train_neg = df_train_all[df_train_all.target == 0]
print(f"No. of positive training examples = {len(df_train_pos)}")
print(f"No. of negative training examples = {len(df_train_neg)}")
train_keywords_unique = df_train_all.keyword.unique()
print(f"No. of unique keywords = {len(train_keywords_unique)}")
df_train_notnull_keywords = df_train_all[~df_train_all.keyword.isnull()]
print(f"No of train examples with keyword not null = {len(df_train_notnull_keywords)}")

No. of positive training examples = 3271
No. of negative training examples = 4342
No. of unique keywords = 222
No of train examples with keyword not null = 7552


# Feature Engineering
## NLP libraries and functions

In [ ]:
# import re
# import nltk
# import gensim
# import sys
# import spacy
# import emoji
# import warnings
# import en_core_web_sm
# from spacy import displacy
# from spacy.tokenizer import Tokenizer
# from tqdm import tqdm
# import seaborn as sns
# import string
# from collections import defaultdict
# from collections import Counter
# from nltk.corpus import stopwords
# from nltk.corpus import stopwords
# from nltk.corpus import wordnet
# from nltk.stem.snowball import SnowballStemmer
# import gensim.corpora as corpora
# from gensim.utils import simple_preprocess
# from spacy.lang.char_classes import CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
# from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER
# from spacy.util import compile_infix_regex
# from emoji import demojize
# from bs4 import BeautifulSoup
# from html import unescape
# import torch
# from transformers import AutoModel, AutoTokenizer 
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# unicode = str

# special = string.punctuation 
# warnings.filterwarnings("ignore")
# nltk.download('stopwords')
# nltk.download('wordnet')
# tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# stemmer = SnowballStemmer("english")
# nlp = spacy.load('en_core_web_lg')

# # Tweet symbols
# # Retrieve the default token-matching regex pattern
# re_token_match = spacy.tokenizer._get_regex_pattern(nlp.Defaults.token_match)
# # Add #hashtag pattern
# re_token_match = f"({re_token_match}|#\\w+)"
# nlp.tokenizer.token_match = re.compile(re_token_match).match

# # Punctuations I want to remove, including the empty token
# puncts = ['\u200d','?', '....','..','...','', ',', '.', '"', ':', ')', '(', '-', '!', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '*', '+', '\\', 
#     '•', '~', '£', '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', 
#     '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', 
#     '—', '‹', '─', '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 
#     'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', 
#     '¹', '≤', '‡', '√', '!','🅰','🅱']

# disaster_words = ['injur',
#  'crust',
#  'battle',
#  'terrorism',
#  'impact',
#  'bush%20fires',
#  'inundated',
#  'rogue',
#  'hostages',
#  'theft',
#  'heat',
#  'oil%20spill',
#  'dust%20storm',
#  'burned',
#  'wreck',
#  'siren',
#  'rockfall',
#  'boat',
#  'nuclear',
#  'infestation',
#  'lightning',
#  'burning%20buildings',
#  'rubble',
#  'life',
#  'rioting',
#  'killing',
#  'traumatised',
#  'emergency%20plan',
#  'bombing',
#  'bermuda',
#  'thunderstorm',
#  'drowning',
#  'fatal',
#  'harm',
#  'outbreak',
#  'collide',
#  'disease',
#  'catastrophe',
#  'disaster',
#  'structural%20failure',
#  'riverine',
#  'threat',
#  'seiche',
#  'apocalypse',
#  'shooting',
#  'hurricane',
#  'obliterate',
#  'weather',
#  'fog',
#  'buildings%20burning',
#  'violent',
#  'sea',
#  'smoke',
#  'drought',
#  'cyclone',
#  'blazing',
#  'hazard',
#  'pyroclastic',
#  'deluge',
#  'hijacking',
#  'conditions',
#  'blew%20up',
#  'poverty',
#  'armageddon',
#  'ruin',
#  'rape',
#  'panicking',
#  'flood',
#  'sandstorm',
#  'cold',
#  'violent%20storm',
#  'blown%20up',
#  'panic',
#  'destroy',
#  'wrecked',
#  'cloud',
#  'volcano',
#  'ocean',
#  'deluged',
#  'crushed',
#  'bombed',
#  'tragedy',
#  'emergency',
#  'ice',
#  'snow',
#  'damage',
#  'curfew',
#  'floods',
#  'mass%20murderer',
#  'demolish',
#  'surge',
#  'detonate',
#  'hijack',
#  'suicide%20bomb',
#  'winter',
#  'collapsed',
#  'wild%20fires',
#  'injuries',
#  'blaze',
#  'lava',
#  'rescuers',
#  'electrocute',
#  'hail',
#  'suicide%20bombing',
#  'death',
#  'richter',
#  'crashed',
#  'whirlwind',
#  'weapons',
#  'obliterated',
#  'volcanic',
#  'wind',
#  'hot',
#  'frost',
#  'flooding',
#  'sinkhole',
#  'drown',
#  'sea-level',
#  'stretcher',
#  'collided',
#  'avalanche',
#  'trauma',
#  'grasshopper',
#  'mudflow',
#  'demolished',
#  'weapon',
#  'loud%20bang',
#  'ambulance',
#  'explosion',
#  'flames',
#  'collision',
#  'dead',
#  'severe',
#  'spill',
#  'chilly',
#  'freeze',
#  'robbery',
#  'fire%20truck',
#  'inundation',
#  'suicide%20bomber',
#  'rain',
#  'dust',
#  'wounded',
#  'mayhem',
#  'attacked',
#  'epidemic',
#  'locust',
#  'radiation%20emergency',
#  'annihilated',
#  'survived',
#  'body%20bag',
#  'landslide',
#  'hostage',
#  'collapse',
#  'pandemic',
#  'storm',
#  'evacuate',
#  'injury',
#  'desolation',
#  'quarantined',
#  'stuck',
#  'danger',
#  'dam',
#  'meltdown',
#  'screams',
#  'annihilation',
#  'obliteration',
#  'refugees',
#  'forest%20fires',
#  'coastal',
#  'brush',
#  'arsonist',
#  'terrorist',
#  'nimbus',
#  'monster',
#  'casualty',
#  'famine',
#  'emergency%20services',
#  'sirens',
#  'derailment',
#  'destruction',
#  'body%20bags',
#  'permafrost',
#  'pandemonium',
#  'massacre',
#  'explode',
#  'bioterror',
#  'blight',
#  'evacuated',
#  'derailed',
#  'thunder',
#  'sink',
#  'tremor',
#  'police',
#  'viral',
#  'casualties',
#  'rescue',
#  'attack',
#  'airplane%20accident',
#  'war%20zone',
#  'hijacker',
#  'military',
#  'drowned',
#  'crash',
#  'heat%20wave',
#  'typhoon',
#  'blow',
#  'cliff',
#  'burning',
#  'shelter',
#  'iceberg',
#  'bloody',
#  'ash',
#  'engulfed',
#  'tsunami',
#  'whirlpool',
#  'debris',
#  'desolate',
#  'destroyed',
#  'airburst',
#  'hellfire',
#  'bush',
#  'bacterial',
#  'chemical%20emergency',
#  'detonat',
#  'detonation',
#  'aftershock',
#  'bleeding',
#  'bioterrorism',
#  'first%20responders',
#  'earthquake',
#  'razed',
#  'mudslide',
#  'glacial',
#  'deaths',
#  'pasture',
#  'rescued',
#  'cumulonimbus',
#  'rainstorm',
#  'virus',
#  'quarantine',
#  'blood',
#  'bridge%20collapse',
#  'nuclear%20disaster',
#  'seismic',
#  'forest',
#  'crush',
#  'eyewitness',
#  'derail',
#  'sinking',
#  'flash',
#  'snowstorm',
#  'exploded',
#  'demolition',
#  'fungal',
#  'wave',
#  'buildings%20on%20fire',
#  'wreckage',
#  'trapped',
#  'injured',
#  'natural%20disaster',
#  'lahar',
#  'screaming',
#  'wounds',
#  'mountain',
#  'geyser',
#  'forest%20fire',
#  '911',
#  'prion',
#  'erosion',
#  'screamed',
#  'bomb',
#  'temperature',
#  'epicentre',
#  'windstorm',
#  'blackout',
#  'trouble',
#  'fatality',
#  'shockwave',
#  'wildfire',
#  'survive',
#  'riot',
#  'gale',
#  'cliff%20fall',
#  'nuclear%20reactor',
#  'arson',
#  'ablaze',
#  'fear',
#  'electrocuted',
#  'twister',
#  'body%20bagging',
#  'displaced',
#  'flattened',
#  'army',
#  'hailstorm',
#  'gust',
#  'fatalities',
#  'upheaval',
#  'sunk',
#  'blizzard',
#  'hazardous',
#  'fire',
#  'accident',
#  'parasitic',
#  'catastrophic',
#  'stabbing',
#  'devastation',
#  'fault',
#  'devastated',
#  'tornado',
#  'derecho',
#  'geomagnetic',
#  'mass%20murder',
#  'magma',
#  'evacuation',
#  'sand',
#  'survivors']

# chat_words = {
#     "AFAIK": "As Far As I Know",
#     "AFK": "Away From Keyboard",
#     "ASAP": "As Soon As Possible",
#     "ATK": "At The Keyboard",
#     "ATM": "At The Moment",
#     "A3": "Anytime, Anywhere, Anyplace",
#     "BAK": "Back At Keyboard",
#     "BBL": "Be Back Later",
#     "BBS": "Be Back Soon",
#     "BFN": "Bye For Now",
#     "B4N": "Bye For Now",
#     "BRB": "Be Right Back",
#     "BRT": "Be Right There",
#     "BTW": "By The Way",
#     "B4": "Before",
#     "B4N": "Bye For Now",
#     "CU": "See You",
#     "CUL8R": "See You Later",
#     "CYA": "See You",
#     "FAQ": "Frequently Asked Questions",
#     "FC": "Fingers Crossed",
#     "FWIW": "For What It's Worth",
#     "FYI": "For Your Information",
#     "GAL": "Get A Life",
#     "GG": "Good Game",
#     "GN": "Good Night",
#     "GMTA": "Great Minds Think Alike",
#     "GR8": "Great!",
#     "G9": "Genius",
#     "IC": "I See",
#     "ICQ": "I Seek you (also a chat program)",
#     "ILU": "ILU: I Love You",
#     "IMHO": "In My Honest/Humble Opinion",
#     "IMO": "In My Opinion",
#     "IOW": "In Other Words",
#     "IRL": "In Real Life",
#     "KISS": "Keep It Simple, Stupid",
#     "LDR": "Long Distance Relationship",
#     "LMAO": "Laugh My A.. Off",
#     "LOL": "Laughing Out Loud",
#     "LTNS": "Long Time No See",
#     "L8R": "Later",
#     "MTE": "My Thoughts Exactly",
#     "M8": "Mate",
#     "NRN": "No Reply Necessary",
#     "OIC": "Oh I See",
#     "PITA": "Pain In The A..",
#     "PRT": "Party",
#     "PRW": "Parents Are Watching",
#     "ROFL": "Rolling On The Floor Laughing",
#     "ROFLOL": "Rolling On The Floor Laughing Out Loud",
#     "ROTFLMAO": "Rolling On The Floor Laughing My Ass Off",
#     "SK8": "Skate",
#     "STATS": "Your sex and age",
#     "ASL": "Age, Sex, Location",
#     "THX": "Thank You",
#     "TTFN": "Ta-Ta For Now!",
#     "TTYL": "Talk To You Later",
#     "U": "You",
#     "U2": "You Too",
#     "U4E": "Yours For Ever",
#     "WB": "Welcome Back",
#     "WTF": "What The F...",
#     "WTG": "Way To Go!",
#     "WUF": "Where Are You From?",
#     "W8": "Wait",
#     "IMMA": "I am going to",
#     "2NITE": "tonight",
#     "DMED": "mesaged",
#     'DM': "message",
#     "SMH": "I am dissapointed"
# }

# # Thanks to https://stackoverflow.com/a/43023503/3971619
# contractions = {
#     "ain't": "are not",
#     "aren't": "are not",
#     "can't": "cannot",
#     "can't've": "cannot have",
#     "'cause": "because",
#     "could've": "could have",
#     "couldn't": "could not",
#     "couldn't've": "could not have",
#     "didn't": "did not",
#     "doesn't": "does not",
#     "don't": "do not",
#     "hadn't": "had not",
#     "hadn't've": "had not have",
#     "hasn't": "has not",
#     "haven't": "have not",
#     "he'd": "he would",
#     "he'd've": "he would have",
#     "he'll": "he will",
#     "he'll've": "he shall have / he will have",
#     "he's": "he is",
#     "how'd": "how did",
#     "how'd'y": "how do you",
#     "how'll": "how will",
#     "how's": "how is",
#     "i'd": "I would",
#     "i'd've": "I would have",
#     "i'll": "I will",
#     "i'll've": "I will have",
#     "i'm": "I am",
#     "i've": "I have",
#     "isn't": "is not",
#     "it'd": "it would",
#     "it'd've": "it would have",
#     "it'll": "it will",
#     "it'll've": "it will have",
#     "it's": "it is",
#     "let's": "let us",
#     "ma'am": "madam",
#     "mayn't": "may not",
#     "might've": "might have",
#     "mightn't": "might not",
#     "mightn't've": "might not have",
#     "must've": "must have",
#     "mustn't": "must not",
#     "mustn't've": "must not have",
#     "needn't": "need not",
#     "needn't've": "need not have",
#     "o'clock": "of the clock",
#     "oughtn't": "ought not",
#     "oughtn't've": "ought not have",
#     "shan't": "shall not",
#     "sha'n't": "shall not",
#     "shan't've": "shall not have",
#     "she'd": "she would",
#     "she'd've": "she would have",
#     "she'll": "she will",
#     "she'll've": "she will have",
#     "she's": "she is",
#     "should've": "should have",
#     "shouldn't": "should not",
#     "shouldn't've": "should not have",
#     "so've": "so have",
#     "so's": "so is",
#     "that'd": "that had",
#     "that'd've": "that would have",
#     "that's": "that is",
#     "there'd": "there would",
#     "there'd've": "there would have",
#     "there's": "there is",
#     "they'd": "they would",
#     "they'd've": "they would have",
#     "they'll": "they will",
#     "they're": "they are",
#     "they've": "they have",
#     "to've": "to have",
#     "wasn't": "was not",
#     "we'd": "we would",
#     "we'd've": "we would have",
#     "we'll": "we will",
#     "we'll've": "we will have",
#     "we're": "we are",
#     "we've": "we have",
#     "weren't": "were not",
#     "what'll": "what will",
#     "what're": "what are",
#     "what's": "what is",
#     "what've": "what have",
#     "when's": "when is",
#     "when've": "when have",
#     "where'd": "where did",
#     "where's": "where is",
#     "where've": "where have",
#     "who'll": "who will",
#     "who's": "who is",
#     "who've": "who have",
#     "why's": "why is",
#     "why've": "why have",
#     "will've": "will have",
#     "won't": "will not",
#     "won't've": "will not have",
#     "would've": "would have",
#     "wouldn't": "would not",
#     "wouldn't've": "would not have",
#     "y'all": "you all",
#     "y'all'd": "you all would",
#     "y'all'd've": "you all would have",
#     "y'all're": "you all are",
#     "y'all've": "you all have",
#     "you'll": "you will",
#     "you're": "you are",
#     "you've": "you have",
# }

# EMOTICONS = {
#     u":‑\)":"Happy face or smiley",
#     u":\)":"Happy face or smiley",
#     u":-\]":"Happy face or smiley",
#     u":\]":"Happy face or smiley",
#     u":-3":"Happy face smiley",
#     u":3":"Happy face smiley",
#     u":->":"Happy face smiley",
#     u":>":"Happy face smiley",
#     u"8-\)":"Happy face smiley",
#     u":o\)":"Happy face smiley",
#     u":-\}":"Happy face smiley",
#     u":\}":"Happy face smiley",
#     u":-\)":"Happy face smiley",
#     u":c\)":"Happy face smiley",
#     u":\^\)":"Happy face smiley",
#     u"=\]":"Happy face smiley",
#     u"=\)":"Happy face smiley",
#     u":‑D":"Laughing, big grin or laugh with glasses",
#     u":D":"Laughing, big grin or laugh with glasses",
#     u"8‑D":"Laughing, big grin or laugh with glasses",
#     u"8D":"Laughing, big grin or laugh with glasses",
#     u"X‑D":"Laughing, big grin or laugh with glasses",
#     u"XD":"Laughing, big grin or laugh with glasses",
#     u"=D":"Laughing, big grin or laugh with glasses",
#     u"=3":"Laughing, big grin or laugh with glasses",
#     u"B\^D":"Laughing, big grin or laugh with glasses",
#     u":-\)\)":"Very happy",
#     u":‑\(":"Frown, sad, andry or pouting",
#     u":-\(":"Frown, sad, andry or pouting",
#     u":\(":"Frown, sad, andry or pouting",
#     u":‑c":"Frown, sad, andry or pouting",
#     u":c":"Frown, sad, andry or pouting",
#     u":‑<":"Frown, sad, andry or pouting",
#     u":<":"Frown, sad, andry or pouting",
#     u":‑\[":"Frown, sad, andry or pouting",
#     u":\[":"Frown, sad, andry or pouting",
#     u":-\|\|":"Frown, sad, andry or pouting",
#     u">:\[":"Frown, sad, andry or pouting",
#     u":\{":"Frown, sad, andry or pouting",
#     u":@":"Frown, sad, andry or pouting",
#     u">:\(":"Frown, sad, andry or pouting",
#     u":'‑\(":"Crying",
#     u":'\(":"Crying",
#     u":'‑\)":"Tears of happiness",
#     u":'\)":"Tears of happiness",
#     u"D‑':":"Horror",
#     u"D:<":"Disgust",
#     u"D:":"Sadness",
#     u"D8":"Great dismay",
#     u"D;":"Great dismay",
#     u"D=":"Great dismay",
#     u"DX":"Great dismay",
#     u":‑O":"Surprise",
#     u":O":"Surprise",
#     u":‑o":"Surprise",
#     u":o":"Surprise",
#     u":-0":"Shock",
#     u"8‑0":"Yawn",
#     u">:O":"Yawn",
#     u":-\*":"Kiss",
#     u":\*":"Kiss",
#     u":X":"Kiss",
#     u";‑\)":"Wink or smirk",
#     u";\)":"Wink or smirk",
#     u"\*-\)":"Wink or smirk",
#     u"\*\)":"Wink or smirk",
#     u";‑\]":"Wink or smirk",
#     u";\]":"Wink or smirk",
#     u";\^\)":"Wink or smirk",
#     u":‑,":"Wink or smirk",
#     u";D":"Wink or smirk",
#     u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
#     u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
#     u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
#     u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
#     u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
#     u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
#     u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
#     u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
#     u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
#     u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
#     u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
#     u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
#     u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
#     u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
#     u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
#     u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
#     u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
#     u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
#     u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
#     u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
#     u":‑\|":"Straight face",
#     u":\|":"Straight face",
#     u":$":"Embarrassed or blushing",
#     u":‑x":"Sealed lips or wearing braces or tongue-tied",
#     u":x":"Sealed lips or wearing braces or tongue-tied",
#     u":‑#":"Sealed lips or wearing braces or tongue-tied",
#     u":#":"Sealed lips or wearing braces or tongue-tied",
#     u":‑&":"Sealed lips or wearing braces or tongue-tied",
#     u":&":"Sealed lips or wearing braces or tongue-tied",
#     u"O:‑\)":"Angel, saint or innocent",
#     u"O:\)":"Angel, saint or innocent",
#     u"0:‑3":"Angel, saint or innocent",
#     u"0:3":"Angel, saint or innocent",
#     u"0:‑\)":"Angel, saint or innocent",
#     u"0:\)":"Angel, saint or innocent",
#     u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
#     u"0;\^\)":"Angel, saint or innocent",
#     u">:‑\)":"Evil or devilish",
#     u">:\)":"Evil or devilish",
#     u"\}:‑\)":"Evil or devilish",
#     u"\}:\)":"Evil or devilish",
#     u"3:‑\)":"Evil or devilish",
#     u"3:\)":"Evil or devilish",
#     u">;\)":"Evil or devilish",
#     u"\|;‑\)":"Cool",
#     u"\|‑O":"Bored",
#     u":‑J":"Tongue-in-cheek",
#     u"#‑\)":"Party all night",
#     u"%‑\)":"Drunk or confused",
#     u"%\)":"Drunk or confused",
#     u":-###..":"Being sick",
#     u":###..":"Being sick",
#     u"<:‑\|":"Dump",
#     u"\(>_<\)":"Troubled",
#     u"\(>_<\)>":"Troubled",
#     u"\(';'\)":"Baby",
#     u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
#     u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
#     u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
#     u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
#     u"\(-_-\)zzz":"Sleeping",
#     u"\(\^_-\)":"Wink",
#     u"\(\(\+_\+\)\)":"Confused",
#     u"\(\+o\+\)":"Confused",
#     u"\(o\|o\)":"Ultraman",
#     u"\^_\^":"Joyful",
#     u"\(\^_\^\)/":"Joyful",
#     u"\(\^O\^\)／":"Joyful",
#     u"\(\^o\^\)／":"Joyful",
#     u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
#     u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
#     u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
#     u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
#     u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
#     u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
#     u"\('_'\)":"Sad or Crying",
#     u"\(/_;\)":"Sad or Crying",
#     u"\(T_T\) \(;_;\)":"Sad or Crying",
#     u"\(;_;":"Sad of Crying",
#     u"\(;_:\)":"Sad or Crying",
#     u"\(;O;\)":"Sad or Crying",
#     u"\(:_;\)":"Sad or Crying",
#     u"\(ToT\)":"Sad or Crying",
#     u";_;":"Sad or Crying",
#     u";-;":"Sad or Crying",
#     u";n;":"Sad or Crying",
#     u";;":"Sad or Crying",
#     u"Q\.Q":"Sad or Crying",
#     u"T\.T":"Sad or Crying",
#     u"QQ":"Sad or Crying",
#     u"Q_Q":"Sad or Crying",
#     u"\(-\.-\)":"Shame",
#     u"\(-_-\)":"Shame",
#     u"\(一一\)":"Shame",
#     u"\(；一_一\)":"Shame",
#     u"\(=_=\)":"Tired",
#     u"\(=\^\·\^=\)":"cat",
#     u"\(=\^\·\·\^=\)":"cat",
#     u"=_\^=	":"cat",
#     u"\(\.\.\)":"Looking down",
#     u"\(\._\.\)":"Looking down",
#     u"\^m\^":"Giggling with hand covering mouth",
#     u"\(\・\・?":"Confusion",
#     u"\(?_?\)":"Confusion",
#     u">\^_\^<":"Normal Laugh",
#     u"<\^!\^>":"Normal Laugh",
#     u"\^/\^":"Normal Laugh",
#     u"\（\*\^_\^\*）" :"Normal Laugh",
#     u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
#     u"\(^\^\)":"Normal Laugh",
#     u"\(\^\.\^\)":"Normal Laugh",
#     u"\(\^_\^\.\)":"Normal Laugh",
#     u"\(\^_\^\)":"Normal Laugh",
#     u"\(\^\^\)":"Normal Laugh",
#     u"\(\^J\^\)":"Normal Laugh",
#     u"\(\*\^\.\^\*\)":"Normal Laugh",
#     u"\(\^—\^\）":"Normal Laugh",
#     u"\(#\^\.\^#\)":"Normal Laugh",
#     u"\（\^—\^\）":"Waving",
#     u"\(;_;\)/~~~":"Waving",
#     u"\(\^\.\^\)/~~~":"Waving",
#     u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
#     u"\(T_T\)/~~~":"Waving",
#     u"\(ToT\)/~~~":"Waving",
#     u"\(\*\^0\^\*\)":"Excited",
#     u"\(\*_\*\)":"Amazed",
#     u"\(\*_\*;":"Amazed",
#     u"\(\+_\+\) \(@_@\)":"Amazed",
#     u"\(\*\^\^\)v":"Laughing,Cheerful",
#     u"\(\^_\^\)v":"Laughing,Cheerful",
#     u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
#     u'\(-"-\)':"Worried",
#     u"\(ーー;\)":"Worried",
#     u"\(\^0_0\^\)":"Eyeglasses",
#     u"\(\＾ｖ\＾\)":"Happy",
#     u"\(\＾ｕ\＾\)":"Happy",
#     u"\(\^\)o\(\^\)":"Happy",
#     u"\(\^O\^\)":"Happy",
#     u"\(\^o\^\)":"Happy",
#     u"\)\^o\^\(":"Happy",
#     u":O o_O":"Surprised",
#     u"o_0":"Surprised",
#     u"o\.O":"Surpised",
#     u"\(o\.o\)":"Surprised",
#     u"oO":"Surprised",
#     u"\(\*￣m￣\)":"Dissatisfied",
#     u"\(‘A`\)":"Snubbed or Deflated"
# }
 
# # function to print sentiments
# # of the sentence.
# def sentiment_scores(sentence):
 
#     # Create a SentimentIntensityAnalyzer object.
#     sid_obj = SentimentIntensityAnalyzer()
 
#     # polarity_scores method of SentimentIntensityAnalyzer
#     # object gives a sentiment dictionary.
#     # which contains pos, neg, neu, and compound scores.
#     sentiment_dict = sid_obj.polarity_scores(sentence)
    
#     return sentiment_dict['neg'], sentiment_dict['neu'], sentiment_dict['pos']
        
# # Initializing the lemmatizer
# lemmatizer = nltk.stem.WordNetLemmatizer()

# try:
#     # nlp = spacy.load('en_core_web_sm')
#     nlp = spacy.load('en_core_web_lg') 
#     language_detector = LanguageDetector()
#     nlp.add_pipe(language_detector)
# except BaseException:
#     pass
    
# token_dict = {}

# def normalizeToken(token):
#     lowercased_token = token.lower()
#     if token.startswith("@"):
#         return "@USER"
#     elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
#         return "HTTPURL"
#     elif len(token) == 1:
#         return demojize(token)
#     else:
#         if token == "’":
#             return "'"
#         elif token == "…":
#             return "..."
#         else:
#             return token

# def seed_everything(seed):
#     random.seed(seed)
#     np.random.seed(seed)
    
# def concat_df(train_data, test_data):
#     return pd.concat([train_data, test_data], sort=True).reset_index(drop=True)

# def List_of_words(df): 
#     words = [word for tweet in tqdm(df['text']) for word in tweet.split()]
#     return words

# def List_of_tweets(df):
#     tweets = [tweet for tweet in tqdm(df['text']) ]
#     return tweets

# def mislabeled_tweets(df) : # function that returns mislabeled labeled tweets
#     df = df.groupby(['text']).nunique().sort_values(by='target', ascending=False)
#     df = df[df['target'] > 1]['target']
#     return (df.index.tolist()) 

# # Correct mislabeled tweets
# def correcting_labels(df) : 
#     df['target_relabeled'] = df['target'].copy() 
#     df.loc[df['text'] == 'like for the music video I want some real action shit like burning buildings and police chases not some weak ben winston shit', 'target_relabeled'] = 0
#     df.loc[df['text'] == 'Hellfire is surrounded by desires so be careful and donÛªt let your desires control you! #Afterlife', 'target_relabeled'] = 0
#     df.loc[df['text'] == 'To fight bioterrorism sir.', 'target_relabeled'] = 0
#     df.loc[df['text'] == '.POTUS #StrategicPatience is a strategy for #Genocide; refugees; IDP Internally displaced people; horror; etc. https://t.co/rqWuoy1fm4', 'target_relabeled'] = 1
#     df.loc[df['text'] == 'CLEARED:incident with injury:I-495  inner loop Exit 31 - MD 97/Georgia Ave Silver Spring', 'target_relabeled'] = 1
#     df.loc[df['text'] == '#foodscare #offers2go #NestleIndia slips into loss after #Magginoodle #ban unsafe and hazardous for #humanconsumption', 'target_relabeled'] = 0
#     df.loc[df['text'] == 'In #islam saving a person is equal in reward to saving all humans! Islam is the opposite of terrorism!', 'target_relabeled'] = 0
#     df.loc[df['text'] == 'Who is bringing the tornadoes and floods. Who is bringing the climate change. God is after America He is plaguing her\n \n#FARRAKHAN #QUOTE', 'target_relabeled'] = 1
#     df.loc[df['text'] == 'RT NotExplained: The only known image of infamous hijacker D.B. Cooper. http://t.co/JlzK2HdeTG', 'target_relabeled'] = 1
#     df.loc[df['text'] == "Mmmmmm I'm burning.... I'm burning buildings I'm building.... Oooooohhhh oooh ooh...", 'target_relabeled'] = 0
#     df.loc[df['text'] == "wowo--=== 12000 Nigerian refugees repatriated from Cameroon", 'target_relabeled'] = 0
#     df.loc[df['text'] == "He came to a land which was engulfed in tribal war and turned it into a land of peace i.e. Madinah. #ProphetMuhammad #islam", 'target_relabeled'] = 0
#     df.loc[df['text'] == "Hellfire! We donÛªt even want to think about it or mention it so letÛªs not do anything that leads to it #islam!", 'target_relabeled'] = 0
#     df.loc[df['text'] == "The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.'", 'target_relabeled'] = 0
#     df.loc[df['text'] == "Caution: breathing may be hazardous to your health.", 'target_relabeled'] = 1
#     df.loc[df['text'] == "I Pledge Allegiance To The P.O.P.E. And The Burning Buildings of Epic City. ??????", 'target_relabeled'] = 0
#     df.loc[df['text'] == "#Allah describes piling up #wealth thinking it would last #forever as the description of the people of #Hellfire in Surah Humaza. #Reflect", 'target_relabeled'] = 0
#     df.loc[df['text'] == "that horrible sinking feeling when youÛªve been at home on your phone for a while and you realise its been on 3G this whole time", 'target_relabeled'] = 0
#     df.drop('target', axis= 1, inplace=True) 
#     df.columns = ['id', 'keyword', 'location', 'text', 'target']
#     return df

# def location_binging(df):
#     df['location'].replace({'United States':'USA',
#                             'New York':'USA',
#                               "London":'UK',
#                               "Los Angeles, CA":'USA',
#                               "Washington, D.C.":'USA',
#                               "California":'USA',
#                               "Chicago, IL":'USA',
#                               "Chicago":'USA',
#                               "New York, NY":'USA',
#                               "California, USA":'USA',
#                               "FLorida":'USA',
#                               "Nigeria":'Africa',
#                               "Kenya":'Africa',
#                               "Everywhere":'Worldwide',
#                               "San Francisco":'USA',
#                               "Florida":'USA',
#                               "United Kingdom":'UK',
#                               "Los Angeles":'USA',
#                               "Toronto":'Canada',
#                               "San Francisco, CA":'USA',
#                               "NYC":'USA',
#                               "Seattle":'USA',
#                               "Earth":'Worldwide',
#                               "Ireland":'UK',
#                               "London, England":'UK',
#                               "New York City":'USA',
#                               "Texas":'USA',
#                               "London, UK":'UK',
#                               "Atlanta, GA":'USA',
#                               "Mumbai":"India"},inplace=True)
#     return df

# UNICODE_EMO = emoji.EMOJI_UNICODE_ENGLISH

# emo_vals = ["_".join(i.replace(",","").replace(":","").split()) for i in UNICODE_EMO] + ["_".join(i.replace(",","").replace(":","").split()) for i in UNICODE_EMO]
# emo_vals.sort(key=lambda el: (len(el), el), reverse=True)
# disaster_words.sort(key=lambda el: (len(el), el), reverse=True)

# def convert_emoticons(text):
#     for emot in EMOTICONS:
#         text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").replace(":","").split()), text)
#     return text

# def convert_emojis(text):
#     for emot in UNICODE_EMO:
#         text = re.sub(r'('+emot+')', "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()), text)
#     return text

# def clean_text(text):
#     text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff\xad\x0c6§\\\£\Â*_<>""⎫•{}Γ~]', ' ', str(text))
    
#     text = BeautifulSoup(unescape(text), 'lxml').text
    
#      # Remove html
#     html = re.compile(r'<.*?>')
#     text = html.sub(r'',text)
    
#     # Convert emoticon
#     text = convert_emoticons(text)

#     text = re.sub(r'[\)\(\.\,\;\\\?\&\%\!\+\-]', '', text)
#     if len(text.split("  ")) > 1000:
#         text = " ".join(["".join(w.split(" ")) if len(w.split(' '))>1 else w for w in text.split("  ")])
#     text = re.sub(r'\s', ' ', text)
#     text = re.sub(r"([A-z])\- ([A-z])", r"\1\2", text)
#     text = text.replace('\'','')
#     text = text.replace('. .', '.')
#     text = text.replace('\'','')
#     text = re.sub(r"\s+"," ", text)
    
#     # Convert emoji
#     text = convert_emojis(text)
    
#     # Remove any chunks of consecutive numbers
#     number_strings = re.findall(r'\d+[ \t]\d+', text)
#     ind_num_strings = []
#     for j in number_strings:
#         x = [int(i) for i in j.split()]
#         ind_num_strings.append(x)
    
#     flat_num_list = [item for sublist in ind_num_strings for item in sublist]
    
#     for i in flat_num_list:
#         j=re.sub(r'\d+','',str(i))
#         text = text.replace(str(i),j)

#     texter_filt = re.sub(r"\( ","", text)
#     texter_filt = re.sub(r" \)","", texter_filt)
#     texter_filt = re.sub(r"\(\)","", texter_filt)
#     texter_filt = re.sub(r"\(\s+\)","", texter_filt)
#     texter_filt = re.sub(r" \.",".", texter_filt)
#     texter_filt = re.sub(r"\\.",".", texter_filt)
#     texter_filt = re.sub(r" \,",",", texter_filt)
#     texter_filt = re.sub(r"\\,",",", texter_filt)
#     texter_filt = re.sub(r"\s+\s+"," ", texter_filt)    
#     texter_filt = re.sub(' +', ' ', texter_filt)
#     return texter_filt.lower()

# def word_count(df_all) : 
#     df_all['word_count'] = df_all['text'].apply(lambda x: len(str(x).split()))
#     return df_all

# def unique_word_count(df_all) : 
#     df_all['unique_word_count'] = df_all['text'].apply(lambda x: len(set(str(x).split())))  
#     return df_all

# def stop_word_count(df_all) : 
#     df_all['stop_word_count'] = df_all['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stopwords.words('english')]))
#     return df_all

# def url_count(df_all) : 
#     df_all['url_count'] = df_all['text'].apply(lambda x: ('http' in str(x).lower() or 'https' in str(x).lower() or 'www' in str(x).lower()))
#     return df_all

# def mean_word_length(df_all) : 
#     df_all['mean_word_length'] = df_all['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
#     return df_all

# def char_count(df_all) : 
#     df_all['char_count'] = df_all['text'].apply(lambda x: len(str(x).lower()))
#     return df_all

# def punctuation_count(df_all) : 
#     df_all['punctuation_count'] = df_all['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
#     return df_all

# def hashtag_count(df_all) : 
#     df_all['hashtag_count'] = df_all['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
#     return df_all

# def mention_count(df_all) : 
#     df_all['mention_count'] = df_all['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
#     return df_all

# def emo_count(df_all) : 
#     df_all['emo_count'] = df_all['text'].apply(lambda x: any(c in str(x).lower() for c in emo_vals))
#     return df_all

# def disaster_count(df_all) : 
#     df_all['dis_count'] = df_all['text'].apply(lambda x: any(c in str(x).lower() for c in disaster_words))
#     return df_all

# def stemming(text) : 
#     stemmer = nltk.stem.PorterStemmer()
#     return(" ".join(stemmer.stem(word) for word in text.split()))

# def flatten(l):
#     """
#     Flatten list of lists.
#     """
#     import collections

#     for el in l:
#         if isinstance(
#                 el, collections.Iterable) and not isinstance(
#                 el, (str, bytes)):
#             for ell in flatten(el):
#                 yield ell
#         else:
#             yield el

# infixes = (
#     LIST_ELLIPSES
#     + LIST_ICONS
#     + [
#         r"(?<=[0-9])[+\-\*^](?=[0-9-])",
#         r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
#             al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
#         ),
#         r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
#         # ✅ Commented out regex that splits on hyphens between letters:
#         # r"(?<=[{a}])(?:{h})(?=[{a}])".format(a=ALPHA, h=HYPHENS),
#         r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
#         r'''[-~]'''
#     ]
# )

# def tokenize(sentence, infixes):
#     all_stopwords = spacy.lang.en.stop_words.STOP_WORDS
#     prefix_re = spacy.util.compile_prefix_regex(nlp.Defaults.prefixes)
#     suffix_re = spacy.util.compile_suffix_regex(nlp.Defaults.suffixes)
#     infix_re = compile_infix_regex(infixes)
    
#     def customize_tokenizer(nlp):
#         # Adds support to use `-` as the delimiter for tokenization
#         return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
#                          suffix_search=suffix_re.search,
#                          infix_finditer=infix_re.finditer,
#                          token_match=None
#                         )

#     nlp.tokenizer = customize_tokenizer(nlp)
#     return [token.text for token in sentence if not token.is_stop]    

# def replace_from_dict(x, dic):
#     replaced_counter = 0
#     for item in dic.items():
#         for i, e in enumerate(x):
#             if e == item[0]:
#                 replaced_counter += 1
#                 del x[i]
#                 for ix, token in enumerate(item[1].split()):
#                     x.insert(i + ix,token)
#     return x

# def lemmatize_list(x):
#     x = " ".join(x)
#     # Returning a list again
#     return [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(x)]

# def remove_from_list(x, stuff_to_remove) -> list:
#     for item in stuff_to_remove:
#         # Making sure to iterate through the entire token
#         for i,token in enumerate(x):
#             if item == token:
#                 del x[i]
#     return x

# def get_wordnet_pos(word):
#     tag = nltk.pos_tag([word])[0][1][0].upper()
#     tag_dict = {"J": wordnet.ADJ,
#                     "N": wordnet.NOUN,
#                     "V": wordnet.VERB,
#                     "R": wordnet.ADV}
#     return tag_dict.get(tag, wordnet.NOUN)

# def normalizeTweet(tweet):
#     tokens = tweet.replace("’", "'").replace("…", "...").split(' ')
#     normTweet = " ".join([normalizeToken(token) for token in tokens])

#     normTweet = (
#         normTweet.replace("cannot ", "can not ")
#         .replace("n't ", " n't ")
#         .replace("n 't ", " n't ")
#         .replace("ca n't", "can't")
#         .replace("ai n't", "ain't")
#     )
#     normTweet = (
#         normTweet.replace("'m ", " 'm ")
#         .replace("'re ", " 're ")
#         .replace("'s ", " 's ")
#         .replace("'ll ", " 'll ")
#         .replace("'d ", " 'd ")
#         .replace("'ve ", " 've ")
#     )
#     normTweet = (
#         normTweet.replace(" p . m .", "  p.m.")
#         .replace(" p . m ", " p.m ")
#         .replace(" a . m .", " a.m.")
#         .replace(" a . m ", " a.m ")
#     )

#     return " ".join(normTweet.split())

# def sent_to_topics(text):

#     # text = ' '.join(list(train['text'].values))
#     texter = nlp(text)
#     sentences = list(texter.sents)

#     def sent_to_words(sentences):
#         for sentence in sentences:
#             yield([word for word in tokenize(sentence, infixes)])
    
#     # Tokenize
#     topics = list(flatten(list(sent_to_words(sentences))))
    
#     # Map contractions
#     topics = replace_from_dict(topics, contractions)

#     # Map chat_words
#     topics = replace_from_dict(topics, chat_words)
    
#     # Lemmatize
#     topics = lemmatize_list(topics)
    
#     # Remove punctuation
#     topics = remove_from_list(topics, puncts)
    
#     return topics

# def misspelled_correction(val):
#     for x in val.split(): 
#         if x in miss_corr.keys(): 
#             val = val.replace(x, miss_corr[x]) 
#     return val

## NLP Munging

In [ ]:
# mislabeled_tweets(df_train_all)

In [ ]:
# df_train = correcting_labels(df_train_all)
# df_all = concat_df(df_train_all, df_test_all)

In [ ]:
# df_all = punctuation_count(url_count(location_binging(df_all)))

# misspell_data = pd.read_csv("../input/twitterdisasterclassoutput/aspell.txt",sep=":",names=["correction","misspell"])
# misspell_data.misspell = misspell_data.misspell.str.strip()
# misspell_data.misspell = misspell_data.misspell.str.split(" ")
# misspell_data = misspell_data.explode("misspell").reset_index(drop=True)
# misspell_data.drop_duplicates("misspell",inplace=True)
# miss_corr = dict(zip(misspell_data.misspell, misspell_data.correction))

# df_all['text'] = df_all['text'].apply(lambda x : clean_text(x))
# df_all['text'] = df_all['text'].apply(lambda x : misspelled_correction(x))

In [ ]:
# df_prep = df_all.copy()
# df_all = emo_count(unique_word_count(mention_count(stop_word_count(hashtag_count(df_all)))))
# df_all['text'] = df_all['text'].apply(lambda x : normalizeTweet(x))
# df_all['text'] = df_all['text'].apply(lambda x : sent_to_topics(x))
# df_all['text'] = df_all['text'].apply(lambda x : ' '.join(x).replace('# ', '#').replace('@ ', '@'))
# df_all[['neg', 'neut', 'pos']] = pd.DataFrame(df_all['text'].apply(lambda x : sentiment_scores(x)).values.tolist(), index=df_all.index)
# df_all = disaster_count(char_count(mean_word_length(word_count(df_all))))

# df_all.to_csv('df_nlp.csv', index=False)

In [10]:
df_all = pd.read_csv(f"../input/twitterdisasterclassoutput/df_nlp.csv")
train, test = divide_df(df_all, df_train_all.shape[0])

## Feature embedding

## Naive strategy

In [ ]:
# train_tweet_vectors = None
# test_tweet_vectors = None
# with nlp.disable_pipes():
#     train_tweet_vectors = np.array([nlp(str(row.text)).vector for id, row in pd.DataFrame(train[['id', 'text']]).reset_index(drop=True).iterrows()])
#     test_tweet_vectors = np.array([nlp(str(row.text)).vector for id, row in pd.DataFrame(test[['id', 'text']]).reset_index(drop=True).iterrows()])

## Using Bert Tweet transformer 

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large")
# bertweet = AutoModel.from_pretrained("vinai/bertweet-large")

# def embed_twitter_feats(tokenized):
#     with torch.no_grad():
#         last_hidden_states = bertweet(torch.tensor([tokenized]))
#     return last_hidden_states[0][:,0,:].numpy()

# train = train.dropna(subset=['text'])
# test = test.dropna(subset=['text'])
# train_tokenized = train['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, padding=True)).values.tolist()
# test_tokenized = test['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, padding=True)).values.tolist()

# train_tweet_vectors = []
# for t in train_tokenized:
#     train_tweet_vectors.append(embed_twitter_feats(t))
# train_tweet_vectors = np.array(np.vstack(train_tweet_vectors))

# test_tweet_vectors = []
# for t in test_tokenized:
#     test_tweet_vectors.append(embed_twitter_feats(t))
# test_tweet_vectors = np.array(np.vstack(test_tweet_vectors))

In [ ]:
!pip install simpletransformers
import gc
import torch
from transformers import pipeline
from scipy.special import softmax
import sklearn
from simpletransformers.classification import ClassificationModel, ClassificationArgs
gc.collect()
torch.cuda.empty_cache()

use_cuda=False

# Create a ClassificationModel
model_args = ClassificationArgs(num_train_epochs=2, 
                                overwrite_output_dir=True)
model_args.manual_seed = 42
model_args.normalization = True #this enables the built-in Bertweet custom tokenizer

model_args.reprocess_input_data = True
#odel_args.evaluate_during_training = True
#model_args.evaluate_during_training_verbose = True
model_args.train_batch_size = 80
model_args.eval_batch_size = 80
model_args.early_stopping_metric = "mcc"
model_args.early_stopping_metric_minimize = False
model_args.use_early_stopping = True
model_args.early_stopping_consider_epochs = True
model_args.early_stopping_patience = 1

model = ClassificationModel(model_type='bertweet', 
                            model_name='vinai/bertweet-base', 
                            args = model_args, 
                            num_labels = 2,
                            use_cuda = False)

training_df = train[["text", "target"]]
training_df.columns = ["text", "labels"]

shuffled_training = training_df.sample(frac=1).reset_index(drop=True)

model.train_model(shuffled_training,
                  acc=sklearn.metrics.accuracy_score, 
                  f1=sklearn.metrics.f1_score)

result, model_outputs, wrong_predictions = model.eval_model(shuffled_training, 
                                                            acc=sklearn.metrics.accuracy_score,
                                                            f1=sklearn.metrics.f1_score)

     |████████████████████████████████| 249 kB 4.3 MB/s            
     |████████████████████████████████| 10.1 MB 76.8 MB/s            
     |████████████████████████████████| 325 kB 81.2 MB/s            
     |████████████████████████████████| 43 kB 1.1 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 212 kB 44.6 MB/s            
     |████████████████████████████████| 76 kB 3.5 MB/s             
     |████████████████████████████████| 4.3 MB 61.4 MB/s            
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=204871bb7d892b0237f784c03cd0f1f8c24c9f395da48e8005b9cb05faa02224
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/7613 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/96 [00:00<?, ?it/s]

# Prepare target label vectors

In [ ]:
# other_columns = ['url_count', 'hashtag_count', 'punctuation_count', 'stop_word_count', 'word_count', 'unique_word_count', 'mean_word_length', 'char_count', 'mention_count', 'neg', 'pos', 'neut', 'emo_count', 'dis_count']
# train_other = train[other_columns].reset_index(drop=True)
# X_train_all = pd.concat([pd.DataFrame(train_tweet_vectors), train_other[other_columns]], axis=1)
# test_other = test[other_columns].reset_index(drop=True)
# X_test_all = pd.concat([pd.DataFrame(test_tweet_vectors), test_other], axis=1)
# X_all = pd.concat([X_train_all, pd.DataFrame(df_train_all['target'].values, columns=['target'])], axis=1)
# X_all = X_all.dropna(thresh=90, axis=0)
# X_all = concat_df(X_all, X_test_all)

In [ ]:
# # X_all.to_csv('../input/twitterdisasterclassoutput/X_all_raw.csv', index=False)
# # X_all = pd.read_csv('../input/twitterdisasterclassoutput/X_all_raw.csv')
# # X_all.to_csv('/home/dpys/Documents/Kaggle_Competitions/nlp-getting-started/X_all_raw.csv', index=False)
# X_all = pd.read_csv('/home/dpys/Documents/Kaggle_Competitions/nlp-getting-started/X_all_raw.csv')

In [ ]:
# X_all = X_all.dropna(thresh=90, axis=0)
# X_train_all = X_all.dropna(subset=['target'], axis=0).drop(columns=['target'])
# X_test_all = X_all[X_all['target'].isna()].drop(columns=['target'])
# train_targets = X_all.dropna(subset=['target'], axis=0)['target'].values
# X_all = X_all.drop(columns=['target'])
# X_all[['emo_count', 'dis_count', 'url_count']] = X_all[['emo_count', 'dis_count', 'url_count']].astype('int')

# Preprocess

In [ ]:
# preprocess = FunctionTransformer(preprocess_x_y)
# cleaned = preprocess.fit_transform(X=X_all)

In [ ]:
# X_train_clean = cleaned.head(X_train_all.shape[0])
# X_test_clean = cleaned.tail(X_test_all.shape[0])

# X_train_clean = X_train_clean.reset_index(drop=True)
# X_test_clean = X_test_clean.reset_index(drop=True)

# X_train_clean.to_csv('/kaggle/working/df_train_preprocessed2.csv', index=False)
# X_test_clean.to_csv('/kaggle/working/df_test_preprocessed2.csv', index=False)

# # X_train_clean.to_csv('/home/dpys/Documents/Kaggle_Competitions/nlp-getting-started/df_train_preprocessed2.csv', index=False)
# # X_test_clean.to_csv('/home/dpys/Documents/Kaggle_Competitions/nlp-getting-started/df_test_preprocessed2.csv', index=False)

In [ ]:
# seed=42
# X_train_clean = pd.read_csv('/home/dpys/Documents/Kaggle_Competitions/nlp-getting-started/df_train_preprocessed2.csv')
# X_test_clean = pd.read_csv('/home/dpys/Documents/Kaggle_Competitions/nlp-getting-started/df_test_preprocessed2.csv')

# # X_train_clean = pd.read_csv('/kaggle/working/df_train_preprocessed2.csv')
# # X_test_clean = pd.read_csv('/kaggle/working/df_test_preprocessed2.csv')

# # deriv_features = ['url_count', 'punctuation_count', 'stop_word_count', 'word_count', 'unique_word_count', 'mean_word_length', 'char_count']
# # X_train_deriv, X_test_deriv, y_train_deriv, y_test_deriv = train_test_split(X_train_clean[deriv_features], train_targets, random_state=seed)

# # X_train, X_test, y_train, y_test = train_test_split(X_train_clean.drop(columns=deriv_features), train_targets, random_state=seed)

# X_train, X_test, y_train, y_test = train_test_split(X_train_clean, train_targets, random_state=seed)

# Tuning

In [ ]:
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression

# models = [
#             'SVC',
#             'EN'
#          ]
                
# estimators = [
#         SVC(random_state=seed, kernel='rbf', class_weight='balanced', max_iter=10000), 
#         LogisticRegression(penalty='elasticnet', fit_intercept=True,
#                                                   solver='saga',
#                                                   class_weight='auto',
#                                                   random_state=seed, max_iter=10000)
# ]
    
# params = {
#             models[0]: {'C':[1.25, 1, 0.75], 'tol': [0.01, 0.005, 0.001]},
#             models[1]: {'l1_ratio': [0.75, 0.8, 0.85, 0.9], 'C':[1, 0.1, 0.05, 0.01], 'tol': [0.05, 0.01, 0.005]}
#          }
    
# def get_pca_range(X):
#     from sklearn.decomposition import PCA
#     pca = PCA(n_components = 0.95)
#     pca.fit(X)
#     min_comps = pca.components_.shape[0]

#     pca = PCA(n_components = 0.99)
#     pca.fit(X)
#     max_comps = pca.components_.shape[0]
#     return np.arange(start=np.round(min_comps, -1), stop=np.round(max_comps, -1), step=10) 

# Model Selection with Grid Search 

## Take subset of data to avoid overcomputation

In [ ]:
# seed=42
# X_train = X_train.reset_index(drop=True)
# y_train = pd.DataFrame(y_train).reset_index(drop=True)

# X_train = X_train.head(750)
# y_train = y_train.head(750)

# X_test = X_test.reset_index(drop=True)
# y_test = pd.DataFrame(y_test).reset_index(drop=True)

# X_test = X_test.head(750)
# y_test = y_test.head(750)

In [ ]:
# import os

# n_comps_range = list(get_pca_range(X_train))
# n_comps_range

# os.environ["TOKENIZERS_PARALLELISM"] = "false"
# #unpatch_sklearn()

In [ ]:
# model_factory = {}

# inner_scoring = "f1"

# feature_select = FeatureUnion([('pca', decomposition.PCA(random_state=seed)), 
# #                                ('rte', RandomTreesEmbedding(n_estimators=10, 
# #                                                             max_depth=3, 
# #                                                             random_state=seed)),
#                                ("anova", SelectFwe(f_classif, alpha=0.01))])
            
# for name, estimator in zip(models, estimators):
#     print(name)
#     model_factory[name] = {}
    
#     # Pipeline feature selection (PCA) with model fitting
#     pipe = Pipeline(
#         [
#              ('feature_select', feature_select),
#              (name, estimator),
#         ]
#     )
#     model_params = {}
#     for hyperparam in params[name].keys():
#         model_params[f"{name}__{hyperparam}"] = params[name][hyperparam]
#     model_params[f"feature_select__pca__n_components"] = n_comps_range
#     pipe_grid_cv = GridSearchCV(pipe, model_params, scoring=[inner_scoring], 
#                        refit=Razors.simplify(param='feature_select__pca__n_components', 
#                                              scoring=inner_scoring, rule="se", sigma=1), 
#                        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=seed), n_jobs=-1)
#     pipe_grid_cv.fit(X_train, y_train.values.ravel())
#     model_factory[name]['oos_score'] = cross_val_score(pipe_grid_cv, X_test, y_test.values.ravel(), 
#                                                        scoring='accuracy', 
#                                                        cv=StratifiedKFold(n_splits=10, 
#                                                                           shuffle=True, 
#                                                                           random_state=seed + 1))
#     model_factory[name]['best_params'] = pipe_grid_cv.best_params_
#     model_factory[name]['best_estimator'] = pipe_grid_cv.best_estimator_

# leaderboard = {}
# for mod in model_factory.keys():
#     leaderboard[mod] = np.mean(model_factory[mod]['oos_score'])

# best_estimator_name = max(leaderboard, key=leaderboard.get)

# best_estimator = model_factory[best_estimator_name]['best_estimator']

# model_factory

# best_estimator.fit(X_train_clean, train_targets)

In [ ]:
# outer_best = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# scores = cross_val_score(best_estimator, X_train_clean, train_targets, scoring='accuracy', cv=outer_best, n_jobs=-1, error_score='raise')
# scores
# final_est = best_estimator

## Fit best model to full training data

In [ ]:
# from sklearn import ensemble
# from sklearn.naive_bayes import GaussianNB
# meta_clf = GaussianNB()

# base_models = [('SVC', model_factory['SVC']['best_estimator']), ('EN', model_factory['EN']['best_estimator'])]

# outer_stacked = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# ec = ensemble.StackingClassifier(estimators=base_models, final_estimator=meta_clf, passthrough=False, cv=outer_stacked)
                                               
# ec.fit(X_train_clean, train_targets)

In [ ]:
# scores = cross_val_score(ec, X_train_clean, train_targets, scoring='accuracy', cv=outer_stacked, n_jobs=-1, error_score='raise')
# scores

# Submission

In [ ]:
# y_test_pred = final_est.predict(X_test_all)
# np.save('../input/twitterdisasterclassoutput/y_predicted.npy', y_test_pred)

In [ ]:
y_test_pred = np.load('../input/twitterdisasterclassoutput/y_predicted.npy')
df_submission = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')
df_submission["target"] = y_test_pred.astype(int)
df_submission.to_csv('submission.csv',index=False)
df_submission